In [5]:
import logging
from config import *
import base64
import requests
from datetime import datetime, timedelta
from urllib.parse import urlencode
import pandas as pd
#import pysftp

now = datetime.now()

logging.basicConfig(
    level=logging.INFO,
    format="{asctime} {levelname:<8} {message}",
    style='{',
    filename=LogFileName,#'%slog' % __file__[:-2],
    filemode='a'
)

if 1==1:
    class SpotifyAPI(object):
        access_token = None
        access_token_expires = now
        access_token_did_expire = True
        client_id = None
        client_secret = None
        token_url = "https://accounts.spotify.com/api/token"

        def __init__(self, client_id, client_secret, *args, **kwargs):
            super().__init__(*args, **kwargs)
            self.client_id = client_id
            self.client_secret = client_secret

        def get_client_credentials(self):
            """
            Returns a base64 encoded string
            """
            client_id = self.client_id
            client_secret = self.client_secret
            if client_secret is None or client_id is None:
                raise Exception("You must set client_id and client_secret")
            client_creds = f"{client_id}:{client_secret}"
            client_creds_b64 = base64.b64encode(client_creds.encode())
            return client_creds_b64.decode()

        def get_token_headers(self):
            client_creds_b64 = self.get_client_credentials()
            return {
                "Authorization": f"Basic {client_creds_b64}"
            }

        @staticmethod
        def get_token_data():
            return {
                "grant_type": "client_credentials"
            }

        def perform_auth(self):
            token_url = self.token_url
            token_data = self.get_token_data()
            token_headers = self.get_token_headers()
            r = requests.post(token_url, data=token_data, headers=token_headers)
            if r.status_code not in range(200, 299):
                raise Exception("Could not authenticate client.")
                # return False
            data = r.json()
            access_token = data['access_token']
            expires_in = data['expires_in']  # seconds
            expires = now + timedelta(seconds=expires_in)
            self.access_token = access_token
            self.access_token_expires = expires
            self.access_token_did_expire = expires < now
            return True

        def get_access_token(self):
            token = self.access_token
            expires = self.access_token_expires
            if expires < now:
                self.perform_auth()
                return self.get_access_token()
            elif token is None:
                self.perform_auth()
                return self.get_access_token()
            return token

        def get_resource_header(self):
            access_token = self.get_access_token()
            headers = {
                "Authorization": f"Bearer {access_token}"
            }
            return headers

        def search(self, query, search_type='artist'):  # type
            headers = self.get_resource_header()
            endpoint = "https://api.spotify.com/v1/search"
            data = urlencode({"q": query, "type": search_type.lower()})
            lookup_url = f"{endpoint}?{data}"
            r = requests.get(lookup_url, headers=headers)  # .json()
            if r.status_code not in range(200, 299):
                return {}
            return r.json()

        def get_resource(self, lookup_id, resource_type='artists', version='v1', target='related-artists'):
            endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}/{target}"
            headers = self.get_resource_header()
            r = requests.get(endpoint, headers=headers)
            if r.status_code not in range(200, 299):
                return {}
            return r.json()

        def get_related_artist(self, _id):
            return self.get_resource(_id, resource_type='artists', target='related-artists')

        def get_artist(self, _id):
            return self.get_resource(_id, resource_type='artists', target='albums')



    spotify = SpotifyAPI(client_id, client_secret)


    def getRelatedArtist(artist_name, id):
        # id='4Awgi8rHD631aMJCXLf21D'   ###uncomment for testing
        related_artists = []
        related_artist_cnt = len(spotify.get_related_artist(id)['artists'])
        # print("related_artist_cnt",related_artist_cnt)
        rank = 0
        JSON_Data = spotify.get_related_artist(id)['artists']  # ('4Awgi8rHD631aMJCXLf21D')

        if related_artist_cnt > 0:
            for x in range(0, related_artist_cnt):
                rank = rank + 1
                genres_cnt = 0
                image_cnt = 0
                genres_cnt = len(JSON_Data[x]['genres'])
                image_cnt = len(JSON_Data[x]['images'])

                # derive metadata
                Artist_Name = artist_name
                RelatedArtist_Name = JSON_Data[x]['name']
                External_Urls_Spotify = JSON_Data[x]['external_urls']['spotify']
                Followers_Total = JSON_Data[x]['followers']['total']
                if genres_cnt > 0:
                    Genres_001 = JSON_Data[x]['genres'][0]
                else:
                    Genres_001 = ''

                if genres_cnt > 1:
                    Genres_002 = JSON_Data[x]['genres'][1]
                else:
                    Genres_002 = ''

                if genres_cnt > 2:
                    Genres_003 = JSON_Data[x]['genres'][2]
                else:
                    Genres_003 = ''

                if genres_cnt > 3:
                    Genres_004 = JSON_Data[x]['genres'][3]
                else:
                    Genres_004 = ''

                if genres_cnt > 4:
                    Genres_005 = JSON_Data[x]['genres'][4]
                else:
                    Genres_005 = ''

                ID = JSON_Data[x]['id']

                if image_cnt > 0:
                    Large_image = JSON_Data[x]['images'][0]['url']
                else:
                    Large_image = ''

                if image_cnt > 1:
                    Medium_image = JSON_Data[x]['images'][1]['url']
                else:
                    Medium_image = ''

                if image_cnt > 2:
                    Small_image = JSON_Data[x]['images'][2]['url']
                else:
                    Small_image = ''

                Popularity = JSON_Data[x]['popularity']
                Type = JSON_Data[x]['type']

                artist = [Artist_Name , RelatedArtist_Name, rank, External_Urls_Spotify , Followers_Total
                    , Genres_001, Genres_002, Genres_003, Genres_004, Genres_005
                    , ID
                    , Large_image
                    , Medium_image
                    , Small_image
                    , Popularity, Type]

                related_artists.append(artist)

        return related_artists


    #download Source Artist file
    if 1==1:

        artists = pd.read_csv(SourceFileName, encoding='UTF-16', sep="|", error_bad_lines=False)#   #For testing , nrows=10
        #print(artists.head(10))
        artists_ids_repo = pd.read_csv("artists_ids_repo.csv",  sep=",", error_bad_lines=False)
        #print(artists_ids_repo.head(10))


        df = pd.merge(artists, artists_ids_repo,
                      left_on='Artist_Name',
                      right_on='artist_name',
                      how='left')

        Missing_Artist_ids = df[df['id'].isnull()]
        print("printing artist with missing ids",Missing_Artist_ids)

        Missing_Artist_ids = Missing_Artist_ids[['Artist_Name']].drop_duplicates()
        Missing_Artist_ids['FULL_NAME'] = Missing_Artist_ids['Artist_Name']
        print(len(Missing_Artist_ids))

        print("Starting the process to extract Artists with missing ID's")

        artist_list = []

        for artist in Missing_Artist_ids['FULL_NAME']:
            # print(artist)
            result = spotify.search(artist, search_type='artist')['artists']['items']
            # print(result)
            try:
                if not result:
                    continue
                else:
                    id_val = result[0]['id']
                    artist_name = artist

                    Artist = [artist_name, id_val]
                    print(Artist)
                    artist_list.append(Artist)
            except:
                pass
                print('no data')

        df_artists_missing_ids = pd.DataFrame(artist_list,
                          columns=['artist_name', 'id'])
        LocalFileName = OutputFileName + '.csv'
        FinalOutputFileName = OutputFileName + '_' + now.strftime('%Y-%m-%d') + '.csv'

        #load artists with missing id's to repo csv file.
        df_artists_missing_ids.to_csv('artists_ids_repo.csv', sep=',', index=False, mode='a', header=False)


        artists_ids_repo = pd.read_csv("artists_ids_repo.csv", sep=",", error_bad_lines=False)
        # print(artists_ids_repo.head(10))

        df_artists_ids = pd.merge(artists, artists_ids_repo,
                      left_on='Artist_Name',
                      right_on='artist_name',
                      how='left')


        Full_Related_Artists = []
        print('Starting data load')



b'Skipping line 4982: expected 2 fields, saw 3\nSkipping line 9351: expected 2 fields, saw 3\n'


printing artist with missing ids                           Artist_Name  ID artist_name   id
340                  Amy Booth Walter NaN         NaN  NaN
565                   Art Ascher Rose NaN         NaN  NaN
1014              Bob Mcalister Crime NaN         NaN  NaN
1526    Chapeleiro & Samantha Machado NaN         NaN  NaN
1649                  Chris Hall Jose NaN         NaN  NaN
1902                       CORY MORAN NaN         NaN  NaN
2046           Dana Henderson Blocker NaN         NaN  NaN
2127              Darryl Pompey Lonny NaN         NaN  NaN
2273                Dawn Koller Maria NaN         NaN  NaN
2458                     DJ Francesca NaN         NaN  NaN
2652                Dustin Newman Roy NaN         NaN  NaN
2890                     ERIC BLANTON NaN         NaN  NaN
2965            Estelle Thrift Phelps NaN         NaN  NaN
3162                   Frank Caliendo NaN         NaN  NaN
3261                  Gala Lyons Veil NaN         NaN  NaN
3545                Gru

In [10]:
for index, row in df_artists_ids.iterrows():
    print(row["Artist_Name"], row["id"])
    # time.sleep(.5)

   

!!! 1mmehjf7eHA10uHMisZGJg
$eem 7dGJo4pcD2V6oG8kP0tJRR
*NSYNC 6Ff53KvcvAj5U7Z1vojB5o
10,000 Maniacs 0MBIKH9DjtBkv8O3nS6szj
11th Hour 1HtIwustFdmwPcgO3IuShY
13 Original Broadway Cast 6Qt5HzQMgUqZAIWtuP397i
16B 0u2qG4roqULELVVO9fMgSG
17 Memphis 4vqV8IpWyagD7V8RjMO63g
1951 2IP71LH7CbwddhsEXBI0fy
2 Phases 7EXjxf79ZOgtH3JlCdNDjz
20 Watt Tombstone 4QydkamDuA2eBx2F9UYDTQ
2001ストリングス オーケストラ 3eYcuixB9ooTUPOGoS88V3
20syl 6mA4csYsYvf4Mq02PleZEV
2nd Chapter Of Acts 0Us5KnZKEz3Y5eand0peXC
2-ZZ-Nuff 6XYRyu0CaRac1YGDOEvvzk
3 Grand 4VjgZddih434IDenCD42hN
3 Of A Kind 0WG74hoaVTaNb3hx3yRqb1
33Miles 6wEuQ6YxQqySp78RhpWjRs
38 Special 3zXw2Eh96iTT51pytzHdZi
4 Strings 2a0fbijQhcaj1hOFp5b3id
48th St. Collective 1mbrPxCCWDAO0UJ0p2gaHn
4Him 5F211gZjfoqvH8v7erGdsy
4i20 3zlIIVkWyNrNpeN8lO49aH
4NDY 5QC9ye8G32MusSmQpj3QUM
4th Ave Jones 4A6PpqluJ0epVdoNjoP0dd
5000 Volts 4U9FglMGShioV0yV8qcNFd
5050 EL BARRET 0jIZMr63wfv8RXPCIG3okM
5050 Flow Malandro 77XFXNr9l7yMoIhzCRK9Ta
56# Alley Chaps 4MqAUolFUBdufumlSH6qj9
5acres

Blow My Whistle Baby 476I7Xie87mV629QjiV4A7
Blu 3Fl1V19tmjt57oBdxXKAjJ
Blu & Exile 567CChDKZrtF4rnk2r5JeP
Blue County 6zvltSMSFDRsPDs8IneC1A
Blue Highway 4nnZcSw4lGTxiq3AvYt8Dd
Blue Island Trio 3TP5cmQcwWeU7QuVtg6uOG
Blue Mafia 6WAnMKt8vCuUQuhsAdjMhO
Blue Mitchell 420BtT2Zyze7Eb9zqHcliW
Blue Moon Rising 6ELzKHos2IdIvKHEFCeMLP
Blue Mountain 4S0WYuq4dnunQV6cL3I49i
Blue Rodeo 4M1fxLs3K8DkyCaTEpdfo0
BlueGrace 3Fl1V19tmjt57oBdxXKAjJ
Bluegrass Brothers 3roW79eKro5UqppsWNr8aq
Bluegrass Cwrkot 0tleexFIZDulBHqMoQPfuu
Bluegrass Diamonds 68r6aSayTvaBbziklVZkNT
BlueRidge 11dIVj4EuyOdtN7vEmNYGc
Blunted Vato 15RLdM61VHtknTIYIuNviR
Bmike 1iwTLEeVSDXJxCY9pg0UZz
Bo Bice 2Q1FIPavG8WZF33kqIP3sy
Bo Diddley 2bmixwMZXlkl2sbIbOfviq
Bo do Catarina 71Pou4FoSYhO5g4HTsrpcT
Bo Garza 1eD9JNoxAgE5gDM4AyjpzS
Bo Phillips Band 6Zdrjq5k8dDhRSYLQVGwLc
Bo Thelin 0Cjq22JgURcGNecDtYtSde
Board-Man 1w9k3VBVzpKs0LsauO5jBd
Bob And Dana Kogut 50o9iZJ9gGIwjuzaykJPEC
Bob Baldwin 6URFeH3cWWv6tj2RazL9IP
Bob Benson 7JbGdCjyh5690DV1G

Country 3sDbKMebVH2VYcRSl7u1VC
Country Roland Band 4lHtr3k2oeLN4wqDAU5LSs
Countrybandists 04V04zXXYi4wxXZDvPmYEf
Coup 2 Pression 0tNkvUGdGE2aExOCMbGS0h
Courtney Marie Andrews 1EI0B66miJj5Fl408B7E9H
Courtney Patton 1HSAiHAbhc2kWXSg8LytDV
Courtney Taylor-Taylor 1FWfsykoxCGqtrmHtuKAbu
Cousin 0KpCz7V5XRkqKuM1JDf56O
Covenant Worship 2F2932LEa8ahflK2xDYKWj
Cover Kid 71JAcPfeNMu1clNa9ifPUx
Cowboy Crush 0XG6XVsM2K2g1RV9Y1qlD6
Cowboy Junkies 3CYSRCHfilgR8DSbkCMp5j
Cowboy Troy 3Rd3kr9QoBdHGjD1Hk0tkj
Cows 4ZSzroBNV7HzBDO9aohuF1
Craig Campbell 7tkwiWhGKSuQV8LwPWceWN
Craig Douglas 2B52XDLCjJiYiEjtBeeOD0
Craig Gerdes 1igkY32M066ApBlQbRucxp
Craig Jones 7pJAUUt4kbWueTldz6dD0j
Craig Morgan 6ktyNEnEukKzskQEiXgCFD
CRAIG P. 4ntmrafQBL0Ar8VtOOu5sx
Craig Pruess 4ntmrafQBL0Ar8VtOOu5sx
Craig Sharpe 4ATxDVnCZvyJ5JAobU4CS4
Craig Shoemaker 5NSrLlZ0PEkuxgbplaZgc9
Craig Urquhart 3nSiwaggcdiu6PMNvXct0K
Craig Wayne Boyd 4zt5aHAmGXYBO8FaWAbDGt
Crash Vegas 4ZvzWjIAH25n5YqTRYUpuT
Crazy Horse 0oi7g8NUnlLh5tJvg2y5e3
Craz

ERIC WATSON 5yDZkbiu4ER9hiE5pbqmYn
Eric Weissberg 2dIzGSHkciATTqFH3Xox5g
Eric Wyse 1IYk7Dbm3X8vVAgrK8tNbv
Eric Zimmermann 4rKfgfMTPq1w6by7D3UMXP
ERICK HERNANDEZ 3MOHzV7k6v2kIn10YjYbT9
Erick Turnbull 1H7E1z6lIIdBqt0j50iq2n
Erik Berglund 5YmK8ZCQMv4YdPKn7mJcLB
Erik Dylan 4Lnd7Ix0gpCGpdKjns4gG8
Erik Mongrain 7Mt6u0GfDpAzQnWYGMCIQJ
Erik Reno 4QL7rtAbYIAQ3sIsffoIz0
Erik Rivera 6m2ykIRw55yiqi2Jhilxng
Erika Henningsen 3hjgA2bJfvsFAbHNM60dvG
Erika Jo 2D0ZiDSiXGy2N8V6mZ2TDe
Erika Perdomo 2Jb4ROzv0sYIJfm9D38tlf
Erika Van Pelt 71subqZU6s1SEqjnwBrNDO
Erikka Innes 5t9fjdDBoGhMfjNlfnZJrH
Erin and Neil Harpe 72BRY84kSpQdqncuRU03GR
Erin Bates 5BqP8GSw4yRxyf4ZhG7Arv
ERIN DAVIS 5MVcp2styzdsv2GTzqOiT4
Erin Enderlin 3ZRP1FiHFKaftTFYOMsFxj
Erin Foley 1NlOG2tcJpyJlLcCt4T9kS
Erin McCarley 6Y4bCmUNPDKqc7dHkVvgim
Erin McKeown 715aftFykfb77AGFFs9WDj
Erin Rae 167VlZ0C0ewQbgKexRFcs6
Erin Willett 2tokm0E2rjwdzyaojNHAvQ
Erkslab 2R1Sq9cZOlg3h32mhH9NVd
Erland Cooper 636k3cBTCgdZfXzCj7Cuaa
Erlend Øye 12AnGvqOxseM5VJLy

J.C. Hyke 0DyA78N7VI3EaSzsoZr9Ub
J.D. Crowe 6gfqvidGxCJgbqSaserlkF
J.D. Crowe & The New South 6gfqvidGxCJgbqSaserlkF
J.D. Sumner 6krxSQGvGldXRaoFYNCYpK
J.D. Sumner & The Stamps Quartet 7gtE0S47YqrOJ2ClbqUm6f
J.E. Sawyer 0puMZicGwhpviA490Ho9Ws
J.J. Cale 06nsZ3qSOYZ2hPVIMcr1IN
J.Kells 12erFyWjs4siPITl6M6yJa
Jack County 4KhOcnCW2PptA8tYkuR8FX
Jack DeJohnette 7rDjbKTLlpNYJRWMm7QVxU
Jack Gray 0LvfDU74rGXrEGVWqW9rCw
Jack Greene 14h0UcPJlGYv5Qrjp7m1jQ
Jack Ingram 7HNEfHmDlFofG6YnMt8G7N
Jack Jeffers 2LiftYcwe6VBVnaIb8xH8c
JACK JONES 6eVCRmRG2HcfL4yx77A9WK
Jack Lukeman 0NllPfB1hk8V4QIKxQW4Od
Jack Rabbit Slim 3nDZWi1wEZpaiEEa9jFLwI
Jack Savoretti 7nC05zmJukRGYObQeRgg3x
Jack Toney 4BDuVUVQP1DjdxEwfzK22A
Jack Vallier 2wnHlwpI5kZaXQRGq21UNN
Jackie Caillier 0PRhkfJfdp7HzvdorQlj9w
Jackie Foster 7jVBGlSwERQx85wRC0RNh0
Jackie Greene 5idHaEmJbiGTZ2MBmhmGMV
Jackie Kashian 06FYrXQiix8vRPUzyOPrAb
JACKIE KENNEDY 5sprC3BOlpim8l6tyepfg7
Jackie Lee 25MYbTsDB9aQ70l6VqNKOR
Jackie Martling 5mjKzfoxLzYWLA2Pcn6asw


...

Marina Raye 2hbupDa1HZ6uiSYFB5pTUb
Marino Tirimo 3N7WuZyPkRPBV4oPMAr7lt
Mario Flores 2xUSUDfwQAxQq9WEal8sId
Mario Lanza 6umNMJq7B8UdOVmwPV6PRL
Mario Parga 3GTImPxZOhmAiQATMnM2yC
Marissa Jaret Winokur 4zdCODAJOsWFl6I9H7z1bX
Marissa Sendejas 5wobSakhePH4byci4wIBDI
Marita Günther 6Lv4fJEiw5d3lx23lYmgUs
Mark Allan Atwood & Brimstone 1GgOqjrRiWWvLECIeiMtQq
Mark Bishop 1rB6iNJyZrKajKrOBwTO6k
Mark Blair 5goo6zwHX9nuOcUFXXwSin
Mark Boozer 2P5kZa8tPNQiS3Crsn3L5p
Mark Bradford 1Tle60TKm1dHph5rfjlYUy
Mark Chesnutt 2cNV8vNe2fPFf6OfGOBbSS
Mark Collie 6kMOabD0AtCGJB632jMilU
Mark Cooke 45b4SNzIyLaQPkzHbppeBy
Mark David Manders 70YnLRjaqsLHvtKKsWpdL8
Mark Davies 3wobEJfFivYClcOnWZdD8I
Mark Erelli 7s3wus2jgjwHQXjFXAQfB2
MARK GREENE 1hIEOyKvQb5gzbvWE5FV5G
Mark Gross 2rFkII2YqsSDtbbEPrOywE
MARK HARRISS 06584gHu2mO2eorGZ9ORgu
Mark Hopkins Band 5ugPfvWbaItRAneNGvZpAX
Mark Knopfler 0FI0kxP0BWurTz8cB8BBug
MARK LABBE 3HBwJLezULQehPYwGyqQuO
Mark Leach 3pn9K5aQEC8ayi76Xe1cPs
Mark Linsky 7kEOMdsEmZkTHgA4ibDz4h
M

PAUL ALLEN 5VpUJ2Hm4NOktjClDtkbgx
Paul Baloche 1jH3GuQCPI87UrS0hcScHr
Paul Bley 4HCwj3Nt6DTTmJCpHuQOeM
Paul Bogart 5hkrBHnb9QDOHT7GwDE6v9
Paul Brady 7lauB9o5ZYmU5lTBOw7w8L
Paul Brandt 11Rb0Ax3Q2dZJ7jNlOvxxP
Paul Burch 5QE1IOAlYbgpOCbIlv1r2j
Paul Cardall 7FQRbf8gbKw8KZQZAJWxH2
Paul Carrack 0FFuvdY7fuiuTmHN9unYoz
Paul Cauthen 6yHM0XQEdu9sIlbILMaKBp
Paul Chambers Quintet 42ckVhl96fQSAXsR2Gm1My
PAUL CHAN 0M1UOBJZ9tcKJbrbnVlHZG
Paul Daigle 0cehMo6PeqalrmuBwO9Wuf
Paul Denton 2fKsd61swALXCOUa8q89U2
Paul Denver 3JWdpadgRIyCfXmqvO34Ef
Paul Eason 0pEz1mSMvaLje9j0suHPww
Paul Elliott 3pb9GpiE5J6ptQfIpiEQE7
Paul Emery 1oHrOYQkMVg7OaHvdDAl2R
PAUL FARMER 1lv5P6IsLMteVFUkEAxAgc
Paul Freeman 6KIWAOKwNIWHHDFcwbTa3p
PAUL GARCIA 1VWsiG5i55l2IH73Vpa5By
Paul Gilbert 19sJfp2FK2evlsw46WVhPG
Paul Gonsalves 23hRv3SqLMrj4i3b3tfUu4
Paul Izak 2gsrDFiFvHxKkh8Qh5Zbys
Paul Kelly 0SNWoGaDlrCompmg9rXeNq
Paul Lamb 79kHMZTWVgos5CQUNSIhEX
Paul Laurence Dunbar 0Fqu8QkiUZE09rYdgVnRg0
Paul Maddox 75PhvOvv3IXTutHblrES6T
Paul 

Sean Wilson 7qHjFxT7sp1lH3aekZHtN7
Seany OWE 1dVMk17ONLm4VCpIMA3L1h
Sebastian Stakset 46kV0QPO6PgUjtP69hvbfI
Sebb Junior 0oP6v3SuOC5jPc5pGUQdsD
Sebi Lee 4UYVPi1AczNfFt5yadvv87
Seckond Chaynce 1Scv2VH4bC1sk2jBvp5h1H
Secondhand Ska Kings 0U6COYlcT8f0HYICy7u80K
Secret Agent 23 Skidoo 7zhRp9EH4bF8lJx7wQtj5Q
Secret Nation 0gX4Kuwic0S5GfOp1oBZYC
Sedef Sebüktekin 1dvuibBCx9TnbCKIdOEF4l
Seedhe Maut 2oBG74gAocPMFv6Ij9ykdo
Seiji Toda 153MHQR6rRrQhKSGT4o73L
Selah 5dZiRoY9Vpp6qrxqXNRwPE
Selena Gomez 0C8ZW7ezQVs4URX5aX7Kqx
Selena Gomez & The Scene 6dJeKm76NjfXBNTpHmOhfO
Selfless 7lBh6TVX5fFNRDw4hGrnKe
Semya 1hatwkL9sjz45yLimEcQOS
Sent Back 5Yz87GHNggmOuevkvePWcl
Seonaidh MacIntyre 5Z06lTLINfdsdfTvkXiMfk
Septima Escala nan
Sera Hill 7oqeyOA1hlWDA5e7xaDjli
Serani 22099GssG7Ood0d5UIh2wo
Serdar Şenel 4ofuU2ViBGnHaTk5XytQ7r
Seré Prince Halverson 4M5UQhlqObdd53gfmMcdeU
Serge Matui 2VqTonKthGJNoE3oYzsxzY
Sergei Tarasov 38sOS4cdVQKr0fLd42ah7g
Sergey UpCent 3XThttz7njH32pGXThfSjU
Sergio Contreras 5QYdUMRHeB

The Juke Joint Pimps 5hQhrVUcXs07omyV3lWUAe
The Junebugs 4k4selYu7cXKeWhIBaM9Cw
The Kasketeers 0mlG55I9xONJWMXg8oCmve
The Kays 0xUi3DpEnXgfuNHyR36HWj
The Kendalls 1rFQlnVFNwI3H6QLDL7LFN
The Kentucky Colonels 5aV6tDnBgmezgCSTvL5HnV
The Kentucky Headhunters 2sgj794K3UaW1ApX28ZUhY
The Kinks 1SQRv42e4PjEYfPhS0Tk9E
The Kinky Boyz 4XI2V1OXjErfjBUJWBJUIN
The Kinleys 54WwKfYhkYzFRtynSgtTw2
The Kyle Bennett Band 5L91aUl7xH6F1pfW4SBCFI
The Lacs 23OFz99wX0NDBBwrxthLWU
The Larry Stephenson Band 2dsjv8di6eXDNVsTa6iE6l
The Last Bandoleros 3Zk03qhmFLepNYW2atWkmE
The Last Revel 2PyWfmEeKmRNhQkAJ1zJXw
The Late 0qvAiM8CkwnigZB6RGIhOn
The Laurie Berkner Band 6T2pk5T8c4Wi61x1v84sUa
The Lawmen 0IJGFJxi9BlnVdIEXwXol8
The LeFevres 5MGXtvQkubWszUU03zUPpZ
The Lettermen 3lzlUccNrekC1oFPCjNIOX
The Light the Heat 17EpO9pUubOAhnTopBgQYR
The Lights Of Dragna 1fyPeVnQvtNaUHHt7lrCOD
The Lil Smokies 6qp6tNvcJu2yZuHxqe4Jls
The Lilac Time 1cMcFL2NdeUDYg0OtD1GeA
The Limeybirds 5TiG4eySdcYrC4LnCFLRHc
The Lindsey Family 7w

Yvncc 06a2LUiG8LAtoEgzZWCkiM
Zac Brown Band 6yJCxee7QumYr820xdIsjo
Zach Crowell 2yZT0zo8NPltj0c8KZvJip
Zach Dylan 1QaXEKVnj3c9ddU4uYSG3G
Zach Ellsworth 5l1YHTJZUhgybfCbQuA5RQ
Zach Hendricks 2BQAOzNn0nA5YBESWe9v6C
Zach Seabaugh 3bmlmoASHg6ycjrKV05CP2
Zach Williams 6g10GEtmIVqIQBhPZh4ScQ
Zack Villere 5jLbQGcvxehi2Z6qkUP9Rh
Zakiyah 306aevG1B9ElBVhAEUILoe
Zane Williams 7LjXLho6TAS9wkSfnjd0AX
Zanski 6IVta6osiP9HifjCLIw4cv
Zara Larsson 1Xylc3o4UrD53lo9CvFvVg
Zauntee 7jyr9Co4MKL1iWML1G7vch
ZAYN 5ZsFI1h6hIdQRw2ti0hz81
Zdeněk Mácal 5liuf1nLjfe3npnEe98eNm
Zé Luiz Mazzziotti 064GPYpEOhc0G7z9p0FKPw
Zee Zee Sleeps 4b3MxvtnW1QOwmaDbM8C3Y
Zeke Hoskin 3JBl4Ur6lWOr7cYzKMBql8
Zeki 0ZZDIo31D35xGrXYTodWrL
Zella Lehr 1c9IYVDDHlIq1Bk6zsJCqA
Zendaya 6sCbFbEjbYepqswM1vWjjs
Zephaniah OHora 6kWzLAW2ZUuy22mLrdwZSY
Zev Haber 2DtSkV9YXvJNhsV9Pgvjip
Zexta Alianza 2dMIA9Z9kAwjqHptVnccGX
Ziggydale Zigfreid 4nyXol4f7pVENGn8hEnCBr
Zizi Possi 65yVjvbkC6m9DzQFihrSvL
Zoë Nutt 1LThzBKBFkowNFtrhFmaEj
Zoe Waites 0wUjp6WeYUSd

In [ ]:
if 1==1:
    if 1==1:
            try:
                Artists = getRelatedArtist(artist_list[i][0], artist_list[i][1])  # getRelatedArtist(related_artist[i])
                for Artist in Artists:
                    Full_Related_Artists.append(Artist)
            except:
                pass
        # create dataset
        df = pd.DataFrame(Full_Related_Artists,
                          columns=['Artist_Name', 'RelatedArtist_Name', 'Rank', 'External_Urls_Spotify', 'Followers_Total'
                              , 'Genres_001', 'Genres_002', 'Genres_003', 'Genres_004', 'Genres_005'
                              , 'ID', 'Large_image', 'Medium_image', 'Small_image', 'Popularity', 'Type', ])
        LocalFileName = OutputFileName + '.csv'
        FinalOutputFileName = OutputFileName +'_'+ now.strftime('%Y-%m-%d') + '.csv'
        df.to_csv(LocalFileName, sep=',', index = False)

        print('Operation completed')
    # with pysftp.Connection(host=myHostname, username=myUsername, password=myPassword) as sftp_upload:
    #     print("Upload connection succesfully stablished ... \n")
    if 1==1:
        

        remotepath = r'/Import/Spotify/'+FinalOutputFileName
        localpath = LocalFileName

        # o_var = sftp_upload.put(localpath, remotepath, confirm=False)
        # print('o_var:', o_var)
        # sftp_upload.close()
        print("File uploaded... \n")
    # In[ ]:
    exit(200)

except Exception as e1:
    print('[Error-1]:', e1)
    logging.ERROR("Error Exception Occured: ",exc_info=True)
    logging.critical("Critical Exception Occured Error: ",exc_info=True)
    logging.warning("Warning Exception Occured Error: ",exc_info=True)